---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [39]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [40]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [41]:
import re

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    df_data = []
    town_file = open('university_towns.txt')
    current_state = ''
    for line in town_file.readlines():
        if re.search('\[edit\]', str(line)):
            current_state = line.split('[')[0].strip()
        else:
            df_data.append([current_state, line.split('(')[0].strip()])
    
    df = pd.DataFrame(df_data)
    df.columns = ['State', 'RegionName']
    return df

In [42]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    # Getting the DF trimmed up for use    
    recession_data = pd.read_excel('gdplev.xls', header=4, usecols=[4,5,6])
    recession_data.dropna(how="all", inplace=True)
    recession_data.columns = ['Quarter', 'GDP Current', 'GDP Chained']
    recession_data = recession_data[['Quarter', 'GDP Chained']]
    recession_data.reset_index(inplace=True, drop=True)
    start_index = recession_data[recession_data['Quarter'] == '2000q1'].index[0]
    recession_data.drop(recession_data.index[:start_index], inplace=True)
    recession_data.reset_index(inplace=True, drop=True)
    cleaned_data = recession_data
    
    max_index = cleaned_data.shape[0] - 1
    
    for index, row in cleaned_data.iterrows():
        if index + 1 <= max_index and index >= 1:
            previous_quarter = cleaned_data.iloc[index-1]
            next_quarter = cleaned_data.iloc[index + 1]
            if row['GDP Chained'] - previous_quarter['GDP Chained'] < 0 and next_quarter['GDP Chained'] - row['GDP Chained'] < 0:
                return row['Quarter']
    
    return None

In [43]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    # Getting the DF trimmed up for use    
    recession_data = pd.read_excel('gdplev.xls', header=4, usecols=[4,5,6])
    recession_data.dropna(how="all", inplace=True)
    recession_data.columns = ['Quarter', 'GDP Current', 'GDP Chained']
    recession_data = recession_data[['Quarter', 'GDP Chained']]
    recession_data.reset_index(inplace=True, drop=True)
    start_index = recession_data[recession_data['Quarter'] == '2000q1'].index[0]
    recession_data.drop(recession_data.index[:start_index], inplace=True)
    recession_data.reset_index(inplace=True, drop=True)
    cleaned_data = recession_data
    
    max_index = cleaned_data.shape[0] - 1
    
    recession_start = get_recession_start()
    
    for index, row in cleaned_data.iterrows():
        if row['Quarter'] > recession_start and index >= 2:
            previous_quarter = cleaned_data.iloc[index - 1]
            second_previous_quarter = cleaned_data.iloc[index - 2]
            if row['GDP Chained'] - previous_quarter['GDP Chained'] > 0 and previous_quarter['GDP Chained'] - second_previous_quarter['GDP Chained'] > 0:
                return row['Quarter']
    
    return None

In [44]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    recession_data = pd.read_excel('gdplev.xls', header=4, usecols=[4,5,6])
    recession_data.dropna(how="all", inplace=True)
    recession_data.columns = ['Quarter', 'GDP Current', 'GDP Chained']
    recession_data = recession_data[['Quarter', 'GDP Chained']]
    recession_data.reset_index(inplace=True, drop=True)
    start_index = recession_data[recession_data['Quarter'] == '2000q1'].index[0]
    recession_data.drop(recession_data.index[:start_index], inplace=True)
    recession_data.reset_index(inplace=True, drop=True)
    cleaned_data = recession_data
    
    start = get_recession_start()
    end = get_recession_end()
    
    recession = cleaned_data[(cleaned_data['Quarter'] >= start) & (cleaned_data['Quarter'] <= end)]
    min_index = recession['GDP Chained'].idxmin()
    return recession.loc[min_index]['Quarter']

In [47]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    raw_housing_data = pd.read_csv('City_Zhvi_AllHomes.csv')
    raw_housing_data.replace({'State': states}, inplace=True)
    
    location_df = raw_housing_data[raw_housing_data.columns[:6]]
    date_df = raw_housing_data[raw_housing_data.columns[6:]]
    date_df = date_df.groupby(pd.PeriodIndex(date_df.columns, freq='Q'), axis=1).mean()
    columns = [str(name).replace('Q', 'q') for name in date_df.columns]
    date_df.columns = columns
    
    date_df = date_df[[c for c in date_df if c >= '2000q1']]
    
    final =  pd.merge(left=location_df, right=date_df, right_index=True, left_index=True).set_index(['State', 'RegionName'])
    final.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'], axis=1, inplace=True)
    final.sort_index(inplace=True)
    return final

FileNotFoundError: File b'City_Zhvi_AllHomes.csv' does not exist

In [48]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    #price_ratio=quarter_before_recession/recession_bottom
    
    housing_data = convert_housing_data_to_quarters()
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    start_column = housing_data.columns.get_loc(recession_start)
    quarter_before = housing_data.columns[start_column - 1]
    #hdf[qrt_bfr_rec_start].div(hdf[rec_bottom]
    housing_data['difference'] = housing_data[quarter_before].div(housing_data[recession_bottom])
    housing_data.reset_index(inplace=True)

    university_towns = get_list_of_university_towns()
    
    housing_merged = pd.merge(left=housing_data, right=university_towns, on=['State', 'RegionName'], indicator=True, how="outer")
    
    university_housing = housing_merged[housing_merged['_merge'] == 'both']
    university_housing.dropna(how="all", inplace=True)
    
    non_university_housing = housing_merged[housing_merged['_merge'] == 'left_only']
    non_university_housing.dropna(how="all", inplace=True)

    test_result = ttest_ind(university_housing['difference'].dropna(), non_university_housing['difference'].dropna())
    
    better = 'university town' if university_housing['difference'].mean() < non_university_housing['difference'].mean() else 'non-university town'
    return (True if test_result.pvalue < 0.01 else False, test_result.pvalue, better )